In [ ]:
import cv2
import time
import numpy as np

# 1. Setup Webcam
cap = cv2.VideoCapture(0)

# 2. Allow the camera to warm up (Crucial for auto-exposure/white balance)
print("Camera warming up... Get out of the frame!")
time.sleep(3) 

# 3. Dynamic Resolution Handling (Improvement from Code #1)
# Instead of hardcoding 640x480, we get the actual webcam dimensions.
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 4. Setup Video Writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (width, height))

# 5. Capture the Background (The "Cloak" effect)
print("Capturing background... Please verify the frame is empty.")
background = 0
for i in range(60):
    ret, background = cap.read()
    if not ret:
        continue
background = np.flip(background, axis=1) # Flip background to match the mirror view later

print("Background captured. You can enter the frame now!")

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break
    
    # Flip the image for a mirror view
    img = np.flip(img, axis=1)
    
    # Convert BGR to HSV for better color detection
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # --- Red Color Detection ---
    # Range 1: Lower red
    lower_red1 = np.array([0, 120, 50])
    upper_red1 = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    
    # Range 2: Upper red (red wraps around 180 in HSV)
    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    
    # Combine masks
    mask_final = mask1 + mask2
    
    # --- Refining the Mask ---
    # Remove noise using Morphological operations
    mask_final = cv2.morphologyEx(mask_final, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    mask_final = cv2.morphologyEx(mask_final, cv2.MORPH_DILATE, np.ones((3, 3), np.uint8))
    
    # Create the inverted mask (everything EXCEPT the red cloth)
    mask_inv = cv2.bitwise_not(mask_final)
    
    # --- The Magic ---
    # 1. Segment out the red cloth from the current frame (Current Frame - Red Cloth)
    res1 = cv2.bitwise_and(img, img, mask=mask_inv)
    
    # 2. Fill the red cloth area with the background image (Background + Mask)
    res2 = cv2.bitwise_and(background, background, mask=mask_final)
    
    # 3. Combine them
    finalOutput = cv2.addWeighted(res1, 1, res2, 1, 0)
    
    # Write to file and show
    out.write(finalOutput)
    cv2.imshow("Invisible Cloak", finalOutput)
    
    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
out.release()
cv2.destroyAllWindows()
